# 🧾 View as a summary

In [ ]:
#| default_exp repr_str

In [ ]:
# |hide
from nbdev.showdoc import *
from fastcore.test import test_eq

In [ ]:
#| hide
#| export
import warnings
import torch
from lovely_numpy.utils import np_to_str_common, pretty_str, sparse_join, PRINT_OPTS, ansi_color

In [ ]:
# |hide
torch.manual_seed(42)
randoms = torch.randn(100)

In [ ]:
nasties = randoms[:12].clone()

nasties[0] *= 10000
nasties[1] /= 10000
nasties[3] = float('inf')
nasties[4] = float('-inf')
nasties[5] = float('nan')
nasties = nasties.reshape((2,6))


In [ ]:
# |exporti
def type_to_dtype(t: str) -> torch.dtype:
    "Convert str, e.g. 'float32' to torch.dtype e.g torch.float32"
    
    dtp = vars(torch)[t]
    assert isinstance(dtp, torch.dtype)
    return dtp

In [ ]:
# |hide
test_eq(type_to_dtype('float16'), torch.float16)
test_eq(type_to_dtype('complex64'), torch.complex64)

In [ ]:
# |exporti
dtnames = { type_to_dtype(k): v
                for k,v in {"float32": "",
                            "float16": "f16",
                            "float64": "f64",
                            "uint8": "u8", # torch does not have uint16/32/64
                            "int8": "i8",
                            "int16": "i16",
                            "int32": "i32",
                            "int64": "i64",
                        }.items()
}

def short_dtype(x): return dtnames.get(x.dtype, str(x.dtype)[6:])

In [ ]:
# |hide
test_eq(short_dtype(torch.tensor(1., dtype=torch.float16)), "f16")

In [ ]:
# | exporti

def plain_repr(x: torch.Tensor):
    "Pick the right function to get a plain repr"
    # assert isinstance(x, np.ndarray), f"expected np.ndarray but got {type(x)}" # Could be a sub-class.
    return x._plain_repr() if hasattr(type(x), "_plain_repr") else repr(x)

def plain_str(x: torch.Tensor):
    "Pick the right function to get a plain str."
    # assert isinstance(x, np.ndarray), f"expected np.ndarray but got {type(x)}"
    return x._plain_str() if hasattr(type(x), "_plain_str") else str(x)

In [ ]:
# | exporti
def is_nasty(t: torch.Tensor):
    """Return true of any `t` values are inf or nan"""
    
    # Unlike .min()/.max(), amin/amax do not allocate extra GPU memory.
    t_min = t.amin()
    t_max = t.amax()

    return (t_min.isnan() or t_min.isinf() or t_max.isinf()).item()

In [ ]:
test_eq(is_nasty(torch.tensor([1, 2, float("nan")])), True)
test_eq(is_nasty(torch.tensor([1, 2, float("inf")])), True)
test_eq(is_nasty(torch.tensor([1, 2, 3])), False)

In [ ]:
# |exporti

def torch_to_str_common(t: torch.Tensor,  # Input
                        color=True,       # ANSI color highlighting
                        ) -> str:
    
    amin, amax = t.amin(), t.amax()

    zeros = ansi_color("all_zeros", "grey", color) if amin.eq(0) and amax.eq(0) and t.numel() > 1 else None
    pinf = ansi_color("+Inf!", "red", color) if amax.isposinf() else None
    ninf = ansi_color("-Inf!", "red", color) if amin.isneginf() else None
    nan = ansi_color("NaN!", "red", color) if amin.isnan() else None

    attention = sparse_join([zeros,pinf,ninf,nan])
    numel = f"n={t.numel()}" if t.numel() > 5 and max(t.shape) != t.numel() else None

    summary = None
    if not zeros:
        minmax = f"x∈[{pretty_str(amin)}, {pretty_str(amax)}]" if t.numel() > 2 else None
        meanstd = f"μ={pretty_str(t.mean())} σ={pretty_str(t.std())}" if t.numel() >= 2 else None
        summary = sparse_join([numel, minmax, meanstd])


    return sparse_join([ summary, attention])

In [ ]:
# |exporti

@torch.no_grad()
def to_str(t: torch.Tensor,
            plain: bool=False,
            verbose: bool=False,
            depth=0,
            lvl=0,
            color=None) -> str:

    if plain:
        return plain_repr(t)

    tname = "tensor" if type(t) is torch.Tensor else type(t).__name__.split(".")[-1]
    shape = str(list(t.shape)) if t.ndim else None
    type_str = sparse_join([tname, shape], sep="")
    
    dev = str(t.device) if t.device.type != "cpu" else None
    dtype = short_dtype(t)
    grad_fn = t.grad_fn.name() if t.grad_fn else None
    # PyTorch does not want you to know, but all `grad_fn``
    # tensors actuall have `requires_grad=True`` too.
    grad = "grad" if t.requires_grad else None 
    

    # For complex tensors, just show the shape / size part for now.
    if not t.is_complex():
        color = PRINT_OPTS.color if color is None else color
        if t.is_cpu or is_nasty(t) or not t.is_floating_point():
            common = np_to_str_common(t.detach().cpu().numpy(), color=color, ddof=1)
        else:
            common = torch_to_str_common(t, color=color)

        vals = pretty_str(t.cpu().numpy()) if t.numel() <= 10 else None
        res = sparse_join([type_str, dtype, common, grad, grad_fn, dev, vals])
    else:
        res = plain_repr(t)


    if verbose or t.is_complex():
        res += "\n" + plain_repr(t)

    if depth and t.dim() > 1:
        res += "\n" + "\n".join([
            " "*PRINT_OPTS.indent*(lvl+1) +
            str(StrProxy(t[i,:], depth=depth-1, lvl=lvl+1))
            for i in range(t.shape[0])])

    return res

In [ ]:
# |exporti
def history_warning():
    "Issue a warning (once) ifw e are running in IPYthon with output cache enabled"

    if "get_ipython" in globals() and get_ipython().cache_size > 0:
        warnings.warn("IPYthon has its output cache enabled. See https://xl0.github.io/lovely-tensors/history.html")

In [ ]:
# |hide
get_ipython().cache_size=1000
history_warning()

/tmp/ipykernel_917248/3648473780.py:6: UserWarning: IPYthon has its output cache enabled. See https://xl0.github.io/lovely-tensors/history.html
  warnings.warn("IPYthon has its output cache enabled. See https://xl0.github.io/lovely-tensors/history.html")


In [ ]:
# |hide
get_ipython().cache_size=0

In [ ]:
#| exporti

class StrProxy():
    def __init__(self, t: torch.Tensor, plain=False, verbose=False, depth=0, lvl=0, color=None):
        self.t = t
        self.plain = plain
        self.verbose = verbose
        self.depth=depth
        self.lvl=lvl
        self.color=color
        history_warning()
    
    def __repr__(self):
        return to_str(self.t, plain=self.plain, verbose=self.verbose,
                      depth=self.depth, lvl=self.lvl, color=self.color)

    # This is used for .deeper attribute and .deeper(depth=...).
    # The second onthe results in a __call__.
    def __call__(self, depth=1):
        return StrProxy(self.t, depth=depth)

In [ ]:
# |export
def lovely(t: torch.Tensor, # Tensor of interest
            verbose=False,  # Whether to show the full tensor
            plain=False,    # Just print if exactly as before
            depth=0,        # Show stats in depth
            color=None):    # Force color (True/False) or auto.
    return StrProxy(t, verbose=verbose, plain=plain, depth=depth, color=color)

In [ ]:
print(lovely(randoms[0]))
print(lovely(randoms[:2]))
print(lovely(randoms[:6].view(2, 3))) # More than 2 elements -> show statistics
print(lovely(randoms[:11])) # More than 10 -> suppress data output


TypeError: 'bool' object is not callable

In [ ]:
# |hide
test_eq(str(lovely(randoms[0])), "tensor 1.927")
test_eq(str(lovely(randoms[:2])), "tensor[2] μ=1.707 σ=0.311 [1.927, 1.487]")
test_eq(str(lovely(randoms[:6].view(2, 3))), "tensor[2, 3] n=6 x∈[-2.106, 1.927] μ=0.276 σ=1.594 [[1.927, 1.487, 0.901], [-2.106, 0.678, -1.235]]")
test_eq(str(lovely(randoms[:11])), "tensor[11] x∈[-2.106, 1.927] μ=0.046 σ=1.384")

In [ ]:
grad = torch.tensor(1., requires_grad=True, dtype=torch.float64)
print(lovely(grad)); print(lovely(grad+1))

tensor f64 grad 1.000
tensor f64 grad AddBackward0 2.000


In [ ]:
# |hide
test_eq(str(lovely(grad)), "tensor f64 grad 1.000")
test_eq(str(lovely(grad+1)), "tensor f64 grad AddBackward0 2.000")

In [ ]:
if torch.cuda.is_available():
    print(lovely(torch.tensor(1., device=torch.device("cuda:0"))))
    test_eq(str(lovely(torch.tensor(1., device=torch.device("cuda:0")))), "tensor cuda:0 1.000")

tensor cuda:0 1.000


Do we have __any__ floating point nasties? Is the tensor __all__ zeros?

In [ ]:
# Statistics and range are calculated on good values only, if there are at lest 3 of them.
lovely(nasties)

tensor[2, 6] n=12 x∈[-1.605, 1.927e+04] μ=2.141e+03 σ=6.423e+03 +Inf! -Inf! NaN!

In [ ]:
lovely(nasties, color=False)

tensor[2, 6] n=12 x∈[-1.605, 1.927e+04] μ=2.141e+03 σ=6.423e+03 +Inf! -Inf! NaN!

In [ ]:
lovely(torch.tensor([float("nan")]*11))

tensor[11] NaN!

In [ ]:
lovely(torch.zeros(12))

tensor[12] all_zeros

In [ ]:
lovely(nasties)

tensor[2, 6] n=12 x∈[-1.605, 1.927e+04] μ=2.141e+03 σ=6.423e+03 +Inf! -Inf! NaN!

In [ ]:
test_eq(str(lovely(nasties)),
    'tensor[2, 6] n=12 x∈[-1.605, 1.927e+04] μ=2.141e+03 σ=6.423e+03 \x1b[31m+Inf!\x1b[0m \x1b[31m-Inf!\x1b[0m \x1b[31mNaN!\x1b[0m')
test_eq(str(lovely(torch.tensor([float("nan")]*11))), 'tensor[11] \x1b[31mNaN!\x1b[0m')
test_eq(str(lovely(torch.zeros(12))), 'tensor[12] \x1b[38;2;127;127;127mall_zeros\x1b[0m')

In [ ]:
lovely(torch.tensor([1,2,3], dtype=torch.int32))

tensor[3] i32 x∈[1, 3] μ=2.000 σ=1.000 [1, 2, 3]

In [ ]:
torch.set_printoptions(linewidth=120)
lovely(nasties, verbose=True)

tensor[2, 6] n=12 x∈[-1.605, 1.927e+04] μ=2.141e+03 σ=6.423e+03 +Inf! -Inf! NaN!
tensor([[ 1.9269e+04,  1.4873e-04,  9.0072e-01,         inf,        -inf,         nan],
        [-4.3067e-02, -1.6047e+00, -7.5214e-01,  1.6487e+00, -3.9248e-01, -1.4036e+00]])

In [ ]:
lovely(nasties, plain=True)

tensor([[ 1.9269e+04,  1.4873e-04,  9.0072e-01,         inf,        -inf,         nan],
        [-4.3067e-02, -1.6047e+00, -7.5214e-01,  1.6487e+00, -3.9248e-01, -1.4036e+00]])

In [ ]:
image = torch.load("mysteryman.pt")
image[1,100,100] = float('nan')

lovely(image, depth=1)

tensor[3, 196, 196] n=115248 x∈[-2.118, 2.640] μ=-0.388 σ=1.073 NaN!
  tensor[196, 196] n=38416 x∈[-2.118, 2.249] μ=-0.324 σ=1.036
  tensor[196, 196] n=38416 x∈[-1.966, 2.429] μ=-0.274 σ=0.973 NaN!
  tensor[196, 196] n=38416 x∈[-1.804, 2.640] μ=-0.567 σ=1.178

In [ ]:
# |eval: false

def memstats():
    allocated = int(torch.cuda.memory_allocated() // (1024*1024))
    max_allocated = int(torch.cuda.max_memory_allocated() // (1024*1024))
    return f"Allocated: {allocated} MB, Max: {max_allocated} Mb"

if torch.cuda.is_available():
    cudamem = torch.cuda.memory_allocated()
    print(f"before allocation: {memstats()}")
    numbers = torch.randn((3, 1024, 1024), device="cuda") # 12Mb image
    torch.cuda.synchronize()

    print(f"after allocation: {memstats()}")
    # Note, the return value of lovely() is not a string, but a
    # StrProxy that holds reference to 'numbers'. You have to del
    # any references to it.
    display(lovely(numbers) )
    print(f"after repr: {memstats()}")
    
    del numbers
    # torch.cuda.memory.empty_cache()

    print(f"after cleanup: {memstats()}")
    test_eq(cudamem >= torch.cuda.memory_allocated(), True)

before allocation: Allocated: 0 MB, Max: 0 Mb
after allocation: Allocated: 12 MB, Max: 12 Mb


tensor[3, 1024, 1024] n=3145728 x∈[-5.325, 5.150] μ=-0.000 σ=0.999 cuda:0

after repr: Allocated: 12 MB, Max: 12 Mb
after cleanup: Allocated: 0 MB, Max: 12 Mb


In [ ]:
# We don't really supposed complex numbers yet
c = torch.randn(5, dtype=torch.complex64)
lovely(c)

tensor[5] complex64
tensor([-0.4011-0.4035j,  1.1300+0.0788j, -0.0277+0.9978j, -0.4636+0.6064j, -1.1505-0.9865j])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()